In [ ]:
pip install openai

Import Library and create a client object

In [1]:
from openai import OpenAI

# 🔹 For LM Studio (local)
lmstudio_client = OpenAI(
    base_url="http://localhost:1234/v1",  # LM Studio local API
    api_key="lm-studio"  # dummy key required by library
)

# 🔹 For OpenAI cloud (official API)
# cloud_client = OpenAI(api_key="your-real-openai-key")


Prompt Engineering is the practice of designing and refining inputs (prompts) given to a Large Language Model (LLM) to achieve the desired output. Since LLMs don’t inherently “understand” like humans, the way you phrase your instructions, examples, and context can drastically affect the quality, accuracy, and reliability of the responses.

It’s a mix of art (clear communication) and science (using structured methods).

Zero-Shot Prompting
Definition: Asking the model to perform a task without giving examples.
Use case: When you want the model to generalize directly from the instruction.
Example:
"Translate 'How are you?' into Spanish."

In [2]:
def zero_shot_prompt(prompt, client, model="gemma-3-12b-it"):
    """
    Send a zero-shot prompt using either LM Studio or OpenAI cloud.
    """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=512
    )
    return response.choices[0].message.content


In [ ]:
# Example zero-shot test (LM Studio)
resp = zero_shot_prompt(
    "Translate 'Good morning, how are you?' into French.",
    lmstudio_client,
    model="gemma-3-12b-it"  # match LM Studio model name
)
print(resp)


Few-Shot Prompting
Definition: Provide multiple examples to set a pattern before asking the model to complete a task.
Use case: When you need the model to mimic a style, format, or reasoning pattern.

In [ ]:
def few_shot_prompt(examples, query, client, model="gemma-3-12b-it"):
    messages = [{"role": "system", "content": "You are a helpful assistant."}]
    
    # Add few-shot examples as user/assistant turns
    for ex in examples:
        messages.append({"role": "user", "content": ex["question"]})
        messages.append({"role": "assistant", "content": ex["answer"]})
    
    # Add the actual query at the end
    messages.append({"role": "user", "content": query})

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=512
    )
    return response.choices[0].message.content


In [ ]:
# Few-shot examples
examples = [
    {"question": "Translate 'Good morning' into Spanish.", "answer": "Buenos días."},
    {"question": "Translate 'How are you?' into Spanish.", "answer": "¿Cómo estás?"}
]

# Real query
query = "Translate 'See you later' into Spanish."

# Run with LM Studio client
resp = few_shot_prompt(examples, query, lmstudio_client, model="your-model-name")
print("Response:", resp)


Chain-of-Thought (CoT) Prompting
Definition: Encourage the model to show its reasoning step by step before giving the final answer.
Use case: Useful for math, logic, reasoning-heavy tasks.
Example:
"Solve: 36 ÷ 12 × 3 + 1. Show your reasoning step by step before giving the final answer."

In [6]:
def cot_prompt(prompt, client, model="gemma-3-12b-it", reasoning_instruction=True):
    system_message = "You are a helpful assistant."
    if reasoning_instruction:
        # Encourage step-by-step reasoning
        user_message = (
            f"{prompt}\n\n"
            "Please think step by step before giving the final answer."
        )
    else:
        user_message = prompt

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=512
    )
    return response.choices[0].message.content


In [ ]:
# Math reasoning example
query = "If there are 3 cars and each car has 4 wheels, how many wheels are there in total?"

resp = cot_prompt(query, lmstudio_client, model="your-model-name")
print("Response:\n", resp)


Instruction Prompting
Definition: Clearly state instructions in a directive form.
Use case: Useful for coding, structured outputs, summaries, etc.
Example:
"Write a Python function that reverses a string. Return only the code without explanation."

In [9]:
def instruction_prompt(instruction, client, model="gemma-3-12b-it"):
    """
    Run an instruction-style prompt (direct task prompting).

    Parameters:
        instruction (str): The task or instruction for the model
        client (OpenAI): OpenAI-compatible client (LM Studio or OpenAI)
        model (str): Model name (e.g., "gpt-4o-mini" or LM Studio model)
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant that follows instructions carefully."},
        {"role": "user", "content": instruction}
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=512
    )
    return response.choices[0].message.content


In [ ]:
# Example 1: Summarization
resp1 = instruction_prompt(
    "Summarize the following text in one sentence:\n\nArtificial Intelligence is rapidly advancing and being applied in healthcare, education, and finance.",
    lmstudio_client,
    model="your-model-name"
)
print("Summarization:\n", resp1)

# Example 2: Code generation
resp2 = instruction_prompt(
    "Write a Python function that calculates the factorial of a number.",
    lmstudio_client,
    model="your-model-name"
)
print("Python Code:\n", resp2)


What is LangChain?

LangChain is an open-source framework designed to make it easier to build applications powered by Large Language Models (LLMs) like GPT, Claude, or open-source models (via LM Studio, HuggingFace, etc.).

Instead of just sending a prompt → getting a response, LangChain provides tools, abstractions, and integrations that let you:

Manage prompts (prompt templates, chaining prompts together).

Connect LLMs to external data (databases, APIs, PDFs, websites).

Enable memory (so your chatbot remembers past conversations).

Use reasoning workflows (e.g., break down a task step by step).

Orchestrate agents (let the chatbot call tools or functions when needed).

Think of LangChain as the glue that connects an LLM to real-world applications.

In [ ]:
pip install langchain langchain-openai -U langchain-community


In [12]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory  # ✅ correct import

def create_langchain_chat(model="gemma-3-12b-it", use_lmstudio=True, api_key=None):
    
    # Setup model
    if use_lmstudio:
        llm = ChatOpenAI(
            openai_api_base="http://localhost:1234/v1",
            openai_api_key="lm-studio",  # dummy key
            model=model
        )
    else:
        llm = ChatOpenAI(model=model, openai_api_key=api_key)

    # Prompt template with memory slot
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that remembers context."),
        MessagesPlaceholder("history"),
        ("human", "{input}")
    ])

    chain = prompt | llm

    # Store conversation histories per session
    store = {}
    def get_history(session_id: str):
        if session_id not in store:
            store[session_id] = ChatMessageHistory()
        return store[session_id]

    chat = RunnableWithMessageHistory(
        chain,
        get_history,
        input_messages_key="input",
        history_messages_key="history"
    )

    def chat_fn(message: str, session_id="default"):
        resp = chat.invoke(
            {"input": message},
            config={"configurable": {"session_id": session_id}}
        )
        return resp.content

    return chat_fn


In [13]:
# Create chat function for LM Studio
chat = create_langchain_chat(model="your-model-name", use_lmstudio=True)

print(chat("Hello! Who won the 2018 FIFA World Cup?", session_id="user1"))
print(chat("And who was the top scorer?", session_id="user1"))
print(chat("Summarize what we talked about.", session_id="user1"))


Hi there! France won the 2018 FIFA World Cup! 🏆



Is there anything else I can help you with today?
The top scorer of the 2018 FIFA World Cup was Harry Kane from England! He scored six goals throughout the tournament. ⚽️



Anything else you'd like to know about the World Cup or something different entirely?
Certainly! Here’s a summary of our conversation:

We discussed the 2018 FIFA World Cup. I told you that **France won** and you then asked who the top scorer was. The top scorer for that year was **Harry Kane from England**, with six goals.



Is there anything else I can help clarify or discuss?
